## Tree Based Small Language Model Medical Coding Architecture

The approach is the leverage small language model(s) to traverse a heirarchy of ICD-9 codes and ask many small questions to classify a Note Event from the MIMI-III dataset.

For example, the image below is a representation of a small portion of the ICD9 code tree. The branch in the picture below shows a subset of the 'Infectious and Parasitic Disease' Chapter of the ICD9 code tree. [View a full json representation of the taxonomy here.](./icd9_full.json) For this implementation, ICD-9 Code levels are broken down into Chapters, Blocks, Categories. To expand on this implementation the Tree can be broken down further into Sub-categories, Extension I, and Extension II codes.
  
The coding algorithm recursively walks the tree, starting at the top level and continuing down any branch(es) directed by the mini model until the final codes is returned.

0        Infectious and Parasitic Diseases  
├── 00   Intestinal infectious diseases  
│   ├── 001 Cholera  
│   ├── 002 Typhoid and paratyphoid fevers  
│   ├── 003 Salmonella  
│   ├── 004 Shigellosis  
│   ├── 005 Other poisoning (bacterial)  
│   ├── 006 Amebiasis  
│   ├── 007 Other protozoal intestinal diseases  
│   ├── 008 Intestinal infections due to other organisms  
│   └── 009 Ill-defined intestinal infections  
├── 01   Tuberculosis  
│   ├── 010 Primary tuberculous infection  
│   ├── 011 Pulmonary tuberculosis  
│   ├── 012 Other respiratory tuberculosis  
│   ├── 013 Tuberculosis of meninges and central nervous system  
│   ├── 014 Tuberculosis of intestines, peritoneum, and mesenteric glands  
│   ├── 015 Tuberculosis of bones and joints  
│   ├── 016 Tuberculosis of genitourinary system  
│   ├── 017 Tuberculosis of other organs  
│   ├── 018 Miliary tuberculosis  
│   └── 019 Respiratory tuberculosis unspecified  
...  
├── 09   Rickettsioses and other arthropod-borne diseases  
│   ├── ...   
│   ├── 087 Relapsing fever  
│   └── 088 Other arthropod-borne diseases   

  
This approach is explored further in this paper:  
 [Automated clinical coding using off-the-shelf large language models](https://arxiv.org/pdf/2310.06552) (Boyle et al.)

In [2]:
from src.tree import TaxonomyParser
from zensols.mednlp import ApplicationFactory

from nltk import flatten
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
from textwrap import dedent
from openai import AzureOpenAI
from typing import List, Dict, Any

import pandas as pd
import ast
import functools
import logging
import os


doc_parser = ApplicationFactory.get_doc_parser()
logger = logging.getLogger(__name__)

load_dotenv(find_dotenv(), override=True)
print(os.getenv("AZURE_OPENAI_BASE"))

pd.set_option('display.max_colwidth', None)

/home/zacksoenen/miniconda3/envs/med_code/lib/python3.9/site-packages/medcat/cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


https://medcode-aoai-useast.openai.azure.com/


In [3]:
# Initialize Code Tree
code_tree = TaxonomyParser()
code_tree.read_from_json("icd9_tax.json")

print(code_tree.find_by_name("00"))

Node('/root/0/00', description='Intestinal infectious diseases')


In [4]:
# View Tree
code_tree.visualize("0")

0        Infectious and Parasitic Diseases
├── 00   Intestinal infectious diseases
│   ├── 001 Cholera
│   ├── 002 Typhoid and paratyphoid fevers
│   ├── 003 Salmonella
│   ├── 004 Shigellosis
│   ├── 005 Other poisoning (bacterial)
│   ├── 006 Amebiasis
│   ├── 007 Other protozoal intestinal diseases
│   ├── 008 Intestinal infections due to other organisms
│   └── 009 Ill-defined intestinal infections
├── 01   Tuberculosis
│   ├── 010 Primary tuberculous infection
│   ├── 011 Pulmonary tuberculosis
│   ├── 012 Other respiratory tuberculosis
│   ├── 013 Tuberculosis of meninges and central nervous system
│   ├── 014 Tuberculosis of intestines, peritoneum, and mesenteric glands
│   ├── 015 Tuberculosis of bones and joints
│   ├── 016 Tuberculosis of genitourinary system
│   ├── 017 Tuberculosis of other organs
│   ├── 018 Miliary tuberculosis
│   └── 019 Respiratory tuberculosis unspecified
├── 02   Zoonotic bacterial diseases
│   ├── 020 Plague
│   ├── 021 Tularemia
│   ├── 022 Anthrax

#### Define Helper Functions

In [5]:
# Note Parsing Functions

def format_icd9(x):
    new_codes = []
    code_list = ast.literal_eval(x)
    for code in code_list:
        new_codes.append(f"{code:0>3}".format(num="1"))

    return str(new_codes)

def parse_note(note:str) -> str:
    
    doc = doc_parser(note)

    new_note = set([])
    for tok in doc.tokens:
        if tok.is_concept and tok.tuis_ in ['T184', 'T047', 'T046', 'T033', 'T037','T191','T005', 'T004', 'T007', 'T008']:
            
            # print(tok, tok.detected_name_, tok.sub_names, tok.pref_name_, tok.tuis_, tok.tui_descs_)
            new_note.add(tok.detected_name_.replace("~"," "))
            new_note.add(tok.pref_name_.lower())

    logger.info(f"Note Parsing Complete.")
    
    return " ".join(new_note)

In [6]:
#Scoring Functions

def recall_score(truth, generated):
    actual_list = ast.literal_eval(truth)
    generated_list = ast.literal_eval(generated)

    similar = len(set(actual_list) & set(generated_list))

    return similar / len(actual_list)

def precision_score(truth, generated):
    actual_list = ast.literal_eval(truth)
    generated_list = ast.literal_eval(generated)

    if len(generated_list) == 0:
        return 0

    similar = len(set(actual_list) & set(generated_list))

    return similar / len(generated_list)

def f1_score(truth, generated):
    precision = precision_score(truth, generated)
    recall = recall_score(truth, generated)

    if precision + recall == 0:
        return 0
    else:
        return 2 * (precision * recall) / (precision + recall)

In [7]:
# Make Call to AOAI

def call_aoai(sys:str, prompt:str) -> List:

    aoai_client = AzureOpenAI(
        azure_endpoint = os.getenv("AZURE_OPENAI_BASE"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        api_version="2024-02-01"
    )
    
    response = aoai_client.chat.completions.create(
        model=os.getenv("AZURE_DEPLOYMENT_NAME"), # model = "deployment_name".
        messages=[
            {"role": "system", "content": dedent(sys)},
            {"role": "user", "content": dedent(prompt)}
        ],
    )

    try:
        output = ast.literal_eval(response.choices[0].message.content)
        return output
    except Exception as e:
        logger.warning(f"{e}")
        return []

In [8]:
# Build Prompt Dymanically

def get_options(tree, parent_code):
    children = tree.get_children(parent_code)
    options = []
    for child in children:
        options.append(f"{child.name}: {child.description}")
    
    return '|'.join(options)

def build_prompt(tree, parent_code, note, categories):
    sys = """
    You are a medical expert. Your job is to classify notes of an event into one or more categories. ACCURACY is VERY IMPORTANT to your job.
    Choose the best option(s) based on the categories offered. ALWAYS return at least one index. ONLY choose from categories listed. 
    Respond with a list of quoted string indeces of the categories the note belongs to.
    Think through your answer. 
    
    ### EXAMPLE ###
    Categories = 0: Infectious and Parasitic Diseases | 1: Neoplasms | 2: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders
    Note = Patient has Tuberculosis and an Immunity Disorder
    Answer: ['0','2']
    ## END EXAMPLE ##
    """
    
    
    prompt = f"""
    Categories = {categories}
    Note = {note}
    Answer:
    """

    return sys, prompt

In [9]:
# Recursive Walk of tree and call aoai to get codes

def get_codes_for_note(parent_code, tree, note, level=3):
    
    categories = get_options(tree, parent_code)
    sys, prompt = build_prompt(tree, parent_code, note, categories)

    codes = call_aoai(sys, prompt)
    
    logger.info(f"Parent Code: {parent_code} | Found: {codes}")
    logger.info(f"Prompt: {prompt}")

    if codes == [] or codes == ['']:
        return ['X'*level]
    elif all(len(i) == level for i in codes):
        return codes
    else:
        return list(map(functools.partial(get_codes_for_note, tree=tree, note=note, level=level), codes))
    

## Prepare Data

In [10]:
# df = transform_data("data/") # Only re-run if change in preparation logic
df = pd.read_csv("data/joined/dataset_single_001_088.csv.gz")
print(df.shape)
display(df.dtypes)

(4855, 3)


HADM_ID       int64
TEXT         object
ICD9_CODE    object
dtype: object

In [11]:
# Get L1 and L2 codes for grading purposes

def get_parent_codes(code_tree, codes):
    code_list = ast.literal_eval(codes)
    parent_codes = []
    for code in code_list:
        parent_codes.append(code_tree.find_by_name(code).parent.name)
    
    parent_codes = list(set(parent_codes))
    return str(parent_codes)

df['L2_CODES'] = df['ICD9_CODE'].apply(lambda x: get_parent_codes(code_tree, x))
df['L1_CODES'] = df['L2_CODES'].apply(lambda x: get_parent_codes(code_tree, x))
display(df[['ICD9_CODE', 'L2_CODES', 'L1_CODES']].head(5))

,ICD9_CODE,L2_CODES,L1_CODES
0,['041'],['03'],['0']
1,"['038', '070']","['08', '03']",['0']
2,['041'],['03'],['0']
3,['038'],['03'],['0']
4,['038'],['03'],['0']


In [12]:
# Take Final Subset

df = df[0:10]
print(df.shape)

(10, 5)


In [13]:
# Add Parsed Text field
tqdm.pandas()
df['PARSED_TEXT'] = df['TEXT'].progress_apply(parse_note)

  0%|          | 0/10 [00:00<?, ?it/s]/home/zacksoenen/miniconda3/envs/med_code/lib/python3.9/site-packages/medcat/meta_cat.py:356: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

In [18]:
print(df.shape)
display(df.head(2))

(10, 7)


HADM_ID  \
0   100020   
1   100074   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

: 

## Get ICD9 Codes

### Part 1 - Get Codes from Gpt-4o mini

In [12]:
### SIMPLE TEST ###
"""
res = flatten(get_codes_for_note("root", code_tree, "Tuberculosis of the bones and joints and HIV"))
print(res)
"""
#### END SIMPLE TEST ###

'\nres = flatten(get_codes_for_note("root", code_tree, "Tuberculosis of the bones and joints and HIV"))\nprint(res)\n'

In [1]:
results = []
df['Generated'] = ""
for index, row in df.iterrows():

    # Parse Note
    note = ast.literal_eval(row['TEXT'])[0]
    print(f"Note: {note}")
    # Get Codes
    result = flatten(get_codes_for_note("0", code_tree, note, level=2)) # Change level here if needed

    # Add result to DF
    df.at[index, 'Generated'] = str(result)

NameError: name 'df' is not defined

In [14]:
# View Results

display(df[['ICD9_CODE','L1_CODES','L2_CODES', 'Generated']].head(10))

,ICD9_CODE,L1_CODES,L2_CODES,Generated
0,['041'],['0'],['03'],"['01', '03']"
1,"['038', '070']",['0'],"['03', '08']","['00', '01', '03']"
2,['041'],['0'],['03'],"['01', '03', '04', '08', '05']"
3,['038'],['0'],['03'],"['00', '01', '03']"
4,['038'],['0'],['03'],"['01', '03', '04', '05', '09']"
5,['053'],['0'],['06'],"['01', '00', '03', '04']"
6,['038'],['0'],['03'],['01']
7,"['038', '047']",['0'],"['05', '03']","['01', '03']"
8,"['041', '038']",['0'],['03'],"['01', '03', '04', '05', '08']"
9,['038'],['0'],['03'],"['01', '03', '04', '09']"


## Score Results

#### Grade L2 Output

In [15]:
results = pd.DataFrame()


results['ICD9_CODE'] = df['ICD9_CODE'].apply(format_icd9)
results['Recall'] = df.apply(lambda x: recall_score(x['L2_CODES'], x['Generated']), axis=1)
results['Precision'] = df.apply(lambda x: precision_score(x['L2_CODES'], x['Generated']), axis=1)
results['F1 Score'] = df.apply(lambda x: f1_score(x['L2_CODES'], x['Generated']), axis=1)
display(results[['Recall', 'Precision', 'F1 Score']].mean(axis=0)*100)

Recall       70.000000
Precision    25.166667
F1 Score     34.666667
dtype: float64

#### Grade Final  ICD 9 Code Output

In [24]:
results = pd.DataFrame()

results['ICD9_CODE'] = df['ICD9_CODE'].apply(format_icd9)
results['Recall'] = df.apply(lambda x: recall_score(x['ICD9_CODE'], x['Generated']), axis=1)
results['Precision'] = df.apply(lambda x: precision_score(x['ICD9_CODE'], x['Generated']), axis=1)
results['F1 Score'] = df.apply(lambda x: f1_score(x['ICD9_CODE'], x['Generated']), axis=1)

display(results[['Recall', 'Precision', 'F1 Score']].mean(axis=0)*100)

Recall       5.000000
Precision    3.333333
F1 Score     4.000000
dtype: float64

#### Results Summary

In [ ]:
print(f"Recall = {round(results['Recall'].mean(),2)}")
print(f"Precision = {round(results['Precision'].mean(),2)}")

## Implement Med NLP Note Parsing

In [15]:
results = []
df['Parsed_Generated'] = ""
for index, row in df.iterrows():

    # Parse Note
    note = row['PARSED_TEXT']
    print(f"Note: {note}")

    # Get Codes
    result = flatten(get_codes_for_note("0", code_tree, note, level=2)) # Change level here if needed

    # Add result to DF
    df.at[index, 'Parsed_Generated'] = str(result)

Note: cva communicated uti chf left ventricular hypertrophy leukocytosis congestive heart failure diabetes mellitus, insulin-dependent wheezing chief complaint ground glass opacity hypotension absence of fever urinary tract infection does communicate liver carcinoma afebrile discharge diagnosis chief complaint (finding) hydronephroses wheeze hcc cerebrovascular accident ggo tid hydronephrosis hyponatremia lvh
Note: death (finding) necrotizing fasciitis incisional hernia active inflammation discharge diagnosis chief complaint (finding) chief complaint septic shock blood clot passed away abdominal pain acute inflammation
Note: cva communicated adverse drug reactions hyperkalemia rales acs - acute coronary syndrome acute renal failure chf kidney failure, acute leukocytosis congestive heart failure adverse reaction to drug severe cardiac valve stenosis wheezing chief complaint hypotension cholelithiasis constipated does communicate gestational thyrotoxicosis unstable angina severe stenosis

2024-08-28 13:53:01,804 invalid syntax (<unknown>, line 1)


Note: cva ulcerative colitis respiratory symptom gastric ulcer chorea chronic obstructive pulmonary disease hyperlipidemia uti chf dvts chronic obstructive airway disease congestive heart failure productive cough community acquired pneumonia deep vein thrombosis staphylococcus aureus rheumatic heart disease mass effect palpable mass diabetes mellitus, insulin-dependent staph wheezing chief complaint coughing productive hypotension pneumothorax constipated urinary tract infection gestational thyrotoxicosis masses palpable acquired community pneumonia discharge diagnosis chief complaint (finding) acquired hospital pneumonias respiratory signs symptoms iron deficiency anemia knee pain cerebrovascular accident tid vaginitis gtt nosocomial pneumonia constipation hypogammaglobulinemia abdominal pain pulmonary embolism
Note: left ventricular hypertrophy
Note: hypotension
Note: death (finding) aortic regurgitation adverse drug reactions esophageal cancer acute renal failure bacteria rales pass

2024-08-28 13:53:05,486 invalid syntax (<unknown>, line 1)


Note: acidosis bleeding, intracranial uti streak artifact rhabdomyolyses weight bearing azar kala suicide attempt staphylococcus aureus chronic pain streaking artifact hyperventilation ankylosing spondylitis aspiration pneumonia mass effect c . difficile diabetes mellitus, insulin-dependent staph wheezing chief complaint clostridium difficile (bacteria) pneumonia, ventilator associated hypotension absence of fever urinary tract infection gestational thyrotoxicosis tinea corporis rheumatoid arthritis intracranial hemorrhage afebrile chief complaint (finding) blood clot overdose wheeze unconscious iron deficiency anemia rhabdomyolysis tid gtt room air vap cvl tinea corporis (disorder) unconscious state


In [16]:
display(df[['ICD9_CODE','L1_CODES','L2_CODES', 'Parsed_Generated']].head(10))

,ICD9_CODE,L1_CODES,L2_CODES,Parsed_Generated
0,['041'],['0'],['03'],"['00', '01', '04', '05', '08', '09']"
1,"['038', '070']",['0'],"['08', '03']",['03']
2,['041'],['0'],['03'],"['01', '04']"
3,['038'],['0'],['03'],"['00', '03']"
4,['038'],['0'],['03'],['XX']
5,['053'],['0'],['06'],"['00', '01', '04', '05', '06', '08', '13']"
6,['038'],['0'],['03'],['XX']
7,"['038', '047']",['0'],"['03', '05']",['00']
8,"['041', '038']",['0'],['03'],['XX']
9,['038'],['0'],['03'],"['01', '03', '08']"


In [17]:
results = pd.DataFrame()


results['ICD9_CODE'] = df['ICD9_CODE'].apply(format_icd9)
results['Recall'] = df.apply(lambda x: recall_score(x['L2_CODES'], x['Parsed_Generated']), axis=1)
results['Precision'] = df.apply(lambda x: precision_score(x['L2_CODES'], x['Parsed_Generated']), axis=1)
results['F1 Score'] = df.apply(lambda x: f1_score(x['L2_CODES'], x['Parsed_Generated']), axis=1)
display(results[['Recall', 'Precision', 'F1 Score']].mean(axis=0)*100)

Recall       35.000000
Precision    19.761905
F1 Score     20.833333
dtype: float64